# Reading Experimental Data Files

The experimental data reading API in LISO is very similar to the [EXtra-data](https://extra-data.readthedocs.io/en/latest/) API, which is extensively used by beamline scientists and users at European XFEL. Indeed, the design of data reading API in LISO is inspired by `EXtra-data` and aims to stay as close as possible to it. It is foreseen in the near future, LISO will be able to read a virtual dataset (VDS) consisting of both data from the accelerator and the user sides.

However, there are fundamental differences between `DOOCS` and `Karabo`. Every signal (data) known to `DOOCS` is called a channel. A channel is identified by its unique address consisting of 4 components of the form of `Facility/Device/Location/Property`. While in `Karabo`, the data are categorized into control data identified by `Device/Property` and instrument data identified by `Device/OutputChannel`. The control data is also called "slow data" due to historical reasons, while the instrument data is also known as "fast data" or "pipeline data".

In Karabo, the bottleneck of the data pipeline is the data from bunch-resolved (pulse-resolved) area detectors like AGIPD, LPD and DSSC, which can produce up to thousands of megapixel images per second. To the best of my knowledge, there is no such kind of detectors on the accelerator side.

In [1]:
from liso import open_run

In [2]:
run = open_run("/home/jun/r0004")

run.info()

# of macro pulses:      524

Control channels (8):
  -  XFEL.MAGNETS/MAGNET.ML/QI.63.I1D/KICK_MRAD.SP
  -  XFEL.MAGNETS/MAGNET.ML/QI.64.I1D/KICK_MRAD.SP
  -  XFEL.RF/LLRF.CONTROLLER/VS.A1.I1/AMPL.SAMPLE
  -  XFEL.RF/LLRF.CONTROLLER/VS.A1.I1/PHASE.SAMPLE
  -  XFEL.RF/LLRF.CONTROLLER/VS.AH1.I1/AMPL.SAMPLE
  -  XFEL.RF/LLRF.CONTROLLER/VS.AH1.I1/PHASE.SAMPLE
  -  XFEL.RF/LLRF.CONTROLLER/VS.GUN.I1/AMPL.SAMPLE
  -  XFEL.RF/LLRF.CONTROLLER/VS.GUN.I1/PHASE.SAMPLE

Diagnostic channels (5):
  -  XFEL.DIAG/CAMERA/OTRC.64.I1D/IMAGE_EXT_ZMQ
  -  XFEL.DIAG/CHARGE.ML/TORA.25.I1/CHARGE.ALL
  -  XFEL.SDIAG/BAM/47.I1/LOW_CHARGE.RESOLUTION
  -  XFEL.SDIAG/BAM/47.I1/LOW_CHARGE_SINGLEBUNCH_ARRIVAL_TIME.1
  -  XFEL.SDIAG/BAM/47.I1/SINGLEBUNCH_NUMBER_FOR_ARRIVAL_TIME_HISTORY.1


## Obtain general information

In [3]:
run.pulse_ids[:10]

array([911342622, 911342674, 911342984, 911343000, 911343016, 911343082,
       911343088, 911343218, 911343344, 911343350], dtype=uint64)

In [4]:
run.control_channels

frozenset({'XFEL.MAGNETS/MAGNET.ML/QI.63.I1D/KICK_MRAD.SP',
           'XFEL.MAGNETS/MAGNET.ML/QI.64.I1D/KICK_MRAD.SP',
           'XFEL.RF/LLRF.CONTROLLER/VS.A1.I1/AMPL.SAMPLE',
           'XFEL.RF/LLRF.CONTROLLER/VS.A1.I1/PHASE.SAMPLE',
           'XFEL.RF/LLRF.CONTROLLER/VS.AH1.I1/AMPL.SAMPLE',
           'XFEL.RF/LLRF.CONTROLLER/VS.AH1.I1/PHASE.SAMPLE',
           'XFEL.RF/LLRF.CONTROLLER/VS.GUN.I1/AMPL.SAMPLE',
           'XFEL.RF/LLRF.CONTROLLER/VS.GUN.I1/PHASE.SAMPLE'})

In [5]:
run.diagnostic_channels

frozenset({'XFEL.DIAG/CAMERA/OTRC.64.I1D/IMAGE_EXT_ZMQ',
           'XFEL.DIAG/CHARGE.ML/TORA.25.I1/CHARGE.ALL',
           'XFEL.SDIAG/BAM/47.I1/LOW_CHARGE.RESOLUTION',
           'XFEL.SDIAG/BAM/47.I1/LOW_CHARGE_SINGLEBUNCH_ARRIVAL_TIME.1',
           'XFEL.SDIAG/BAM/47.I1/SINGLEBUNCH_NUMBER_FOR_ARRIVAL_TIME_HISTORY.1'})

## Access a single macropulse data

In [6]:
# From index
pid, data = run.from_index(2)
assert pid == run.pulse_ids[2]

data

{'XFEL.RF/LLRF.CONTROLLER/VS.A1.I1/PHASE.SAMPLE': 1.9274961,
 'XFEL.RF/LLRF.CONTROLLER/VS.AH1.I1/AMPL.SAMPLE': 15.162286,
 'XFEL.RF/LLRF.CONTROLLER/VS.A1.I1/AMPL.SAMPLE': 136.52525,
 'XFEL.MAGNETS/MAGNET.ML/QI.63.I1D/KICK_MRAD.SP': 1065.1,
 'XFEL.RF/LLRF.CONTROLLER/VS.GUN.I1/PHASE.SAMPLE': -47.904385,
 'XFEL.RF/LLRF.CONTROLLER/VS.AH1.I1/PHASE.SAMPLE': -178.22366,
 'XFEL.MAGNETS/MAGNET.ML/QI.64.I1D/KICK_MRAD.SP': -1.248001e-07,
 'XFEL.RF/LLRF.CONTROLLER/VS.GUN.I1/AMPL.SAMPLE': 56.67717,
 'XFEL.DIAG/CAMERA/OTRC.64.I1D/IMAGE_EXT_ZMQ': array([[0, 0, 0, ..., 1, 0, 0],
        [0, 1, 0, ..., 0, 1, 2],
        [0, 0, 1, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 3, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [3, 0, 2, ..., 0, 0, 1]], dtype=uint16),
 'XFEL.SDIAG/BAM/47.I1/SINGLEBUNCH_NUMBER_FOR_ARRIVAL_TIME_HISTORY.1': 1,
 'XFEL.SDIAG/BAM/47.I1/LOW_CHARGE_SINGLEBUNCH_ARRIVAL_TIME.1': 2.9433162,
 'XFEL.SDIAG/BAM/47.I1/LOW_CHARGE.RESOLUTION': 35.056053,
 'XFEL.DIAG/CHARGE.ML/TORA.25.

In [7]:
# From macropulse ID
pid, data = run.from_id(911342984)
assert pid == 911342984

## Access channel data

In [8]:
image_channel = run.channel("XFEL.DIAG/CAMERA/OTRC.64.I1D/IMAGE_EXT_ZMQ")

In [9]:
img_data = image_channel.numpy()

img_data.shape

(524, 1750, 2330)